In [14]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline
plt.style.use('ggplot')


%store -r WORKDIR

if 'WORKDIR' not in dir():
    WORKDIR = 'C:/Users/BZ241WX/Documents/InfNet/CursoPosDataScience/mit_data_science/'


data_raw_file = WORKDIR + '/Data/Raw/covid19_data_modeling.parquet'
data_proc_file = WORKDIR + '/Data/Processed/covid19_data_modeling.parquet'


pd.plotting.register_matplotlib_converters()

# Leitura dos Dados 

In [15]:
data = pd.read_parquet(data_raw_file)

print('shape:', data.shape)
print('columns:', data.columns)
data.head()

shape: (27532, 5)
columns: Index(['cases', 'countrycode', 'date', 'deaths', 'recovered'], dtype='object')


,cases,countrycode,date,deaths,recovered
0,852,AD,6/06/20,51,1
1,852,AD,6/05/20,51,1
2,852,AD,6/04/20,51,1
3,851,AD,6/03/20,51,1
4,844,AD,6/02/20,51,1


# Estatistica dos dados 

In [16]:
data.describe()

,cases,countrycode,date,deaths,recovered
count,27532,27532,27532,27532,27532
unique,10668,190,189,3585,566
top,1,JP,5/31/20,0,0
freq,796,189,189,6149,11254


# Correção do tipo dos dados

In [17]:
data['date'] = pd.to_datetime(data.date, format='%m/%d/%y')

data_types = {
    'cases': float,
    'countrycode': str,
    'deaths': float,
    'recovered': float,
}
for cname, dtype in data_types.items():
    if dtype == float:
        data.loc[data[cname].astype(str) == '',cname] = '0'
    
    data[cname] = data[cname].astype(dtype)
    
data.describe()

,cases,deaths,recovered
count,2.753200e+04,27532.000000,2.753200e+04
mean,3.053033e+04,1588.699695,6.791012e+03
std,1.710954e+05,8395.535039,4.930432e+04
min,1.000000e+00,0.000000,0.000000e+00
25%,7.500000e+01,1.000000,0.000000e+00
50%,8.920000e+02,16.000000,2.000000e+00
75%,7.725750e+03,168.000000,3.800000e+01
max,4.346748e+06,149180.000000,1.089686e+06


# Verificar Valores Nulos

In [18]:
data.isnull().sum()

cases          0
countrycode    0
date           0
deaths         0
recovered      0
dtype: int64

# Variável Alvo 

In [19]:
data.groupby('countrycode').cases.last().sort_values(ascending=False).head(10)

countrycode
US    2732639.0
BR    1368195.0
RU     640246.0
IN     548318.0
GB     300692.0
PE     282365.0
CL     271983.0
ES     248970.0
IR     232863.0
MX     216852.0
Name: cases, dtype: float64

# Exportar Base de Dados 

In [20]:
data.to_parquet(data_proc_file)